![title](./media/walmart_christmas.jpg)

# Walmart: Weihnachtsgeschäft Just in Time

Modulprojekt für die Vorlesung __Data-Mining-Process__

_Dozent:_ Prof. Dr. Johannes Maucher  
_Studenten:_ Stephanie Flohr, Benedikt Haußner, Robert Masendorf

**Hinweis:** Zu korrekten Darstellung der Ergebnisse in den MarkDowns muss die Nbextension ```Python Markdown``` aktiviert sein. Zusätzlich muss ein ```Trusted```Kernel verwendet werden.

Für unser Projekt analysiseren wir Verkaufsdaten der amerikanischen Supermarktkette _Walmart_.

**Vorgehensmodell:** CRISP-DM

<img align="left" src="https://upload.wikimedia.org/wikipedia/commons/b/b9/CRISP-DM_Process_Diagram.png" width="400"/>

## BUSINESS UNDERSTANDING: Ursprung und grober Inhalt des Datensatzes

### Was ist Walmart?
Walmart ist der wertvollste Einzelhändler der Welt mit einer Marktkapitalisierung von 246,2 Mrd. US-Dollar. Weitere Fakten:
+ Anzahl Filialen weltweit: ca 11.600
+ Jahresumsatz 2017: 50,3 Mrd. US-Dollar
+ Gewinn 2017: 9,9 Mrd. US-Dollar
+ Anzahl der Mitarbeiter weltweit: ca 2 Millionen

Quelle: [Wikipedia](https://de.wikipedia.org/wiki/Walmart)

### _Just in Time_ bei Walmart
[Wikipedia](https://de.wikipedia.org/wiki/Walmart)

Walmarts Strategie beruht auf ein Konzept aus geringen Preisen und gewinnmargen begleitet durch geringe Lohn- und Gehaltzahlungen trotz hohem Profit. Ein wichtiger Baustein in diesem Konzept ist auch die Produktion nach Just in Time. Walmart unterhält keine Warenlager, sondern entwickelte eine ausgefeilte Logistik, durch die die Produkte direkt aus der Produktion zu den Logistzentren geliefert werden. Dadurch sind keine Zwischenlager mehr notwendig. Um dieses Konzept weltweit umsetzen zu können, beschäftigt Walmart 2000 Analysten. 

### Wettbewerb auf Kaggle
[Walmart - Nähere Informationen zur Competition](https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting).

Walmart stellt den genutzen Datensatz auf Kaggle für einen Bewerberwettbewerb zur Verfügung. Mit dem Wettbewerb möchte Waltmart die besten Kandidaten für zukünftige Mitarbeiter herausfiltern. Den Bewerbern gibt diese Competition die Möglichkeit, ihr Können vorzuzeigen. 

Der Datensatz beinhaltet historische Verkaufdaten von 45 Walmart Märkten an unterschiedlichen Standorten. Diese sind widerum in sich in unterschiedliche Abteilungen unterteilt. Es gilt je Bereich eine Analyse der Salesdaten zu entwickeln und damit vorhersagen, welcher Bereich am effektivsten arbeitet.

### Ziel unserer Arbeit
Dieses Assignment hat nicht das Ziel, an dem Wettbewerb mitteilzunehmen. Wir haben uns als Ziel gesetzt, aus dem Daten zum Vorweihnachtsgeschäft aus 2010 das Vorweihnachtsgeschäft für 2011 vorhersagen zu können. 

## DATA UNDERSTANDING: Was wissen wir und was wollen wir wissen?

In der zweiten Phase des CRISP-VGMs betrachten wir die uns zur Verfügung stehenden Daten.  
Dabei beantworten wir folgende Fragen:
+ Was sind die Quellen unserer Daten?
+ Wie sind die uns zur Verfügung stehenden Dateien aufgebaut?
+ Welche Wertbereiche umfassen die Daten und welche Verteilungen sind zu erkennen?
    + Gibt es dabei Dinge, die wir so nicht erwartet hätten?
+ Genügt die Qualität der Daten, um in die Phase der Data Preparation übertreten zu können?

### Initiale Datensammlung

In [ ]:
#import sys
#if not sys.warnoptions:
import warnings
warnings.simplefilter("ignore")

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import date
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from keras import layers
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import RMSprop
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

Das von Walmart zur Verfügung gestellte Datenpaket beinhaltet drei für uns relevante csv-Dateien:
+ ```train.csv```
+ ```features.csv```
+ ```stores.csv```

Die beiden Dateien ```test.csv``` und ```sampleSubmission.csv``` sind nur für den Wettbewerb interessant und werden daher hier nicht behandelt.  
Weitere Informationen zu den einzelen Datasets: [Walmart - Data](https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/data)

### Beschreibung der Daten

Im Folgenden werden die Attribute der einzelnen csv-Dateien kurz dargestellt.

#### train.csv

In [ ]:
data_df = pd.read_csv('./data/train.csv')
display(data_df.head())

Die Attribute ```Store```, ```Dept```und ```Date``` sind zusammen der Primärschlüssel der Tabelle.

+ ```Store```: eindeutige Filial-ID
+ ```Dept```: pro Filiale eindeutige Abteilungs-ID
+ ```Date```: Enddatum der Woche, für die die Verkäufe summiert wurden
+ ```Weekly_Sales```: Wochenumsatz pro Abteilung und Filiale
+ ```IsHoliday```: Feiertagsanzeiger
    + ```True```, falls in der vergangenen Woche ein Feiertag lag

#### features.csv

In [ ]:
features_df = pd.read_csv('./data/features.csv')
display(features_df.head())

Die Attribute ```Store``` und ```Date``` sind zusammen der Primärschlüssel der Tabelle.

+ ```Store```: eindeutige Filial-ID
+ ```Date```: Enddatum der Woche
+ ```Temperature```: gemittelte Temperatur der Woche
+ ```Fuel_Price```: gemittelter Benzinpreis der Woche
+ ```MarkDown 1-5```: Metriken, die Auskunft über spezielle Werbeangebote der Woche geben
+ ```CPI```: Verbraucherpreisindex der Woche
+ ```Unemployment```: Arbeitslosenquote der Woche
+ ```IsHoliday```: Feiertagsanzeiger
    + ```True```, falls in der vergangenen Woche ein Feiertag lag
    + Duplikat des Attributs selbigen Namens in der Datei ```train.csv```

#### stores.csv

In [ ]:
stores_df = pd.read_csv('./data/stores.csv')
display(stores_df.head())

+ ```Store```: eindeutige Filial-ID
+ ```Type```: Typ der Filiale (wobei die drei Klassen ```A```, ```B```und ```C``` vorhanden sind)
+ ```Size```: Größe der Filiale
    + Nachdem die Daten anonymisiert sind, sind die Angaben nicht in m<sup>2</sup> oder ft<sup>2</sup> , sondern abstrahiert zu verstehen.

### Datenexploration

Im Folgenden sollen die einzelnen Datensätze getrennt voneinander untersucht werden.  
Von besonderem Interesse sind Wertebereiche, Extremwerte und fehlende Werte.

#### Train - Datensatz

Der Datensatz ```train.csv``` enthält die Attribute ```Store```, ```Dept```, ```Date```, ```Weekly_Sales``` und ```IsHoliday```.

Im Folgenden untersuchen wir die einzelnen Attribute, wobei wir uns auf einzelne, interessante Fragestellungen beschränken, anstatt komplette univariate Analysen auszugeben.  

Das Attribut ```IsHoliday``` wird hier auch von der Anzahl der Abteilungen beeinflusst.  
Nachdem das Attribut auch im Datensatz ```features.csv``` vorkommt, wird es dort betrachtet.

__Store__

Der Datensatz umfasst __{{data_df.Store.nunique()}} einzelne Stores__.

__Dept__

Es gibt insgesamt __{{data_df.Dept.nunique()}} verschiedene Abteilungen__.  
Die IDs der Abteilungen reichen dabei von __{{data_df.Dept.min()}}__ bis __{{data_df.Dept.max()}}__.

Fraglich ist, ob jeder Store auch tatsächlich jede Abteilung beherbergt:

In [ ]:
data_df.groupby('Store')['Dept'].nunique().plot(figsize=(20, 5), kind='bar', rot=0).set_ylabel('Anzahl an Abteilungen')
plt.axhline(data_df.Dept.nunique(), color='red')
plt.ylim(60, data_df.Dept.nunique() + 5)
plt.show()

Wir können beobachten, dass es nicht einen einzigen Store gibt, der alle Abteilungen unter einem Dach vereint.  
__Der kleinste Store umfasst dabei {{data_df.groupby('Store')['Dept'].nunique().min()}} Abteilungen, der größte {{data_df.groupby('Store')['Dept'].nunique().max()}} Abteilungen__.

Weiterhin ist von Interesse, ob es Abteilungen gibt, die in jedem Store vorhanden sind oder eben nicht:

In [ ]:
dept_in_stores_series = data_df.groupby('Dept')['Store'].nunique()
dept_in_stores_series.plot(figsize=(20, 5), kind='bar', rot=0).set_ylabel('Anzahl an Abteilungen')
plt.show()

Insgesamt gibt es __{{dept_in_stores_series.value_counts()[45]}} Abteilungen, die in jedem Store vertreten sind__, das sind ca {{int(dept_in_stores_series.value_counts(normalize=True)[45]*100)}} Prozent aller Abteilungen.  
__Abteilung {{dept_in_stores_series[dept_in_stores_series==1].index.values[0]}} gibt es nur in einem einzigen Store__. Sie ist dabei die einzige Abteilung, die in nur einem Store existiert.

__Date__

Der älteste Messwert ist vom ```{{data_df.Date.min()}}```, der jüngste Messwert ist vom ```{{data_df.Date.max()}}```.  
Insgesamt wurde an {{data_df.Date.nunique()}} Tagen gemessen.

__Weekly Sales__

In [ ]:
print("Anzahl Messpunkte:\t{}".format(data_df.Weekly_Sales.count()))
print("Kleinster Betrag:\t{}".format(data_df.Weekly_Sales.min()))
print("Median:\t\t\t{}".format(data_df.Weekly_Sales.median()))
print("Durchschnitt:\t\t{0:.2f}".format(data_df.Weekly_Sales.mean()))
print("Größter Betrag:\t\t{}".format(data_df.Weekly_Sales.max()))

pd.DataFrame(data_df.Weekly_Sales).boxplot(figsize=(20,1), vert=False)
plt.show()

__Interessant:__ es sind auch negative Werte für den Umsatz enthalten und das nicht zu knapp:  
Dieses Verhalten kann in  1285 Einzelmessungen beobachtet werden, es tritt in allen Stores und Wochen auf.  
Wie es dazu kommt wird [hier](https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/discussion/7152) erklärt, es wurden in der Periode Waren für mehr Wert retourniert als gekauft.

#### Features - Datensatz

Der Datensatz ```features.csv``` enthält die Attribute ```Store```, ```Date```, ```Temperature```, ```Fuel_Price```, ```Markdown 1-5```, ```CPI```, ```Unemployment``` und ```IsHoliday```.

__Store und Date__

Nachdem die Datensätze ```train.csv``` und ```features.csv``` in einem späteren Schritt über die Attribute ```Store``` und ```Date``` gejoined werden sollen, wird hier überprüft, ob ein Inner Join ohne Verluste möglich ist:

In [ ]:
data_join_df = data_df[['Store', 'Date']]

features_join_df = features_df[['Store', 'Date']]
features_join_df = features_join_df[features_join_df.Date <= data_join_df.Date.max()]

cnt_rows_without_partner = len(pd.concat([data_join_df, features_join_df]).drop_duplicates(keep=False))
print("Es wurden für {} Zeilen keine Join-Partner gefunden.".format(cnt_rows_without_partner))

Ein Join ist also ohne Verluste machbar.

__Temperature__

In [ ]:
print("Die niedrigste gemessene Temperatur:\t{}".format(features_df.Temperature.min()))
print("Die höchste gemessene Temperatur:\t{}".format(features_df.Temperature.max()))
print("Median der Temperatur:\t\t\t{}".format(features_df.Temperature.median()))

Nachdem Nachrichten über gekochte Amerikaner auch uns in Europa erreicht hätten, gehen wir davon aus, dass die Temperatur in Fahrenheit vorliegt.  
In einem ersten Schritt wollen wir diese umrechnen:

In [ ]:
def fahr_to_celsius(temp_fahr):
    return round((temp_fahr - 32) * 5 / 9, 2)

features_df['Temperature_F'] = features_df.Temperature
features_df['Temperature_C'] = features_df.Temperature.apply(lambda tmp_fahr: fahr_to_celsius(tmp_fahr))
features_df.drop(columns=['Temperature'], inplace=True)

In [ ]:
print("Die niedrigste gemessene Temperatur:\t{}".format(features_df.Temperature_C.min()))
print("Die höchste gemessene Temperatur:\t{}".format(features_df.Temperature_C.max()))
print("Median der Temperatur:\t\t\t{}".format(features_df.Temperature_C.median()))

Diese Ergebnisse ergeben durchaus mehr Sinn aus der europäischen Perspektive.

In [ ]:
list_of_stores = features_df[features_df.Temperature_C == features_df.Temperature_C.min()].Store.tolist() + features_df[features_df.Temperature_C == features_df.Temperature_C.max()].Store.tolist()
temperature_store_df = pd.DataFrame()

for store in list_of_stores:
    temperature_store_df[store] = features_df.Temperature_C[features_df.Store == store].tolist()
    temperature_store_df.reset_index(drop=True, inplace=True)
    
temperature_store_df.boxplot(figsize=(20,5), vert=False)
plt.xlabel("Temperatur")
plt.ylabel("Store")
plt.show()

__Fuel_Price__

In [ ]:
print("Der niedrigste Preis pro Gallone Sprit:\t\t{}".format(features_df.Fuel_Price.min()))
print("Der höchste Preis pro Gallone Sprit:\t\t{}".format(features_df.Fuel_Price.max()))
print("Median des Spritpreises pro Gallone:\t\t{}".format(features_df.Fuel_Price.median()))

pd.DataFrame(features_df.Fuel_Price).boxplot(figsize=(20,1), vert=False)
plt.show()

__MarkDown 1-5__

Der Beschreibung auf [Kaggle](https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/data) entnehmen wir, dass die Werte für die Marketingaktionen erst ab November 2010 verfügbar sind. Zudem fehlen angeblich Daten für weitere Wochen.  
Nachdem wir die Werte ab September 2010 für die Prognose nutzen wollen, könnte es sein, dass die Attribute für uns wertlos sind.

Auf jeden Fall soll an dieser Stelle untersucht werden, wie es sich mit fehlenden Werten im Datensatz generell verhält:

In [ ]:
pd.DataFrame(features_df.isnull().sum())

Wir wollen wissen, für welchen Zeitraum tatsächlich Daten der Attribute ```MarkDown 1-5``` vorhanden sind:

In [ ]:
print("Erstes Datum mit allen Werten für MarkDown:\t{}".format(features_df[['Date', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].dropna().Date.min()))
print("Letztes Datum mit allen Werten für MarkDown:\t{}".format(features_df[['Date', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']].dropna().Date.max()))

Nachdem wir aus den Daten von 2010 für das Weihnachtsgeschäft von 2011 lernen wollen, sind die Attribute ```MarkDown 1-5``` für uns wertlos und werden in der Phase der __Data Preparation__ entfernt.

__CPI__

Der Verbrauchspreisindex zeigt die Steigerung (oder Minderung) der Lebenshaltungskosten an.  
Dabei existieren verschiedene CPIs - mit welchem genau wir es zu tun haben und welches das Basisjahr ist, konnten wir nicht ermitteln.  

In [ ]:
print("Der niedrigste Wert des CPI:\t{}".format(features_df.CPI.min()))
print("Der höchste Wert des CPI:\t{}".format(features_df.CPI.max()))
print("Median des CPI:\t\t\t{}".format(features_df.CPI.median()))

pd.DataFrame(features_df.CPI).boxplot(figsize=(20,1), vert=False)
plt.show()

Wie soeben in der Auswertung der ```MarkDown```-Attribute gesehen, fehlen auch hier Werte.  
Diese fehlenden Werte müssen, falls nötig, in der Phase der __Data Preparation__ nachberechnet werden.

In [ ]:
cpi_df = features_df.set_index(keys=['Store', 'Date'])['CPI'].unstack(level='Store')
cpi_df.index = pd.to_datetime(cpi_df.index)
cpi_df.plot(figsize=(20,13))
plt.xlabel('Datum')
plt.ylabel('CPI')
plt.show()

__Dieses Finding ist interessant:__  
Warum gibt es Regionen, in denen der CPI den minimalen Wert von ```{{features_df.CPI.min()}}``` hat, im Gegensatz zu Regionen, bei denen der maximale Wert ```{{features_df.CPI.max()}}```, also fast das Doppelte, beträgt?  
Zumindest in Deutschland ließe sich ein derartiger Unterschied im Einzelhandel nicht regional beurteilen.  
Wir werden dieses Thema später eventuell nochmals aufgreifen.

__Unemployment__

In [ ]:
print("Der niedrigste Wert des Unemployment:\t{}".format(features_df.Unemployment.min()))
print("Der höchste Wert des Unemployment:\t{}".format(features_df.Unemployment.max()))
print("Median des Unemployment:\t\t{}".format(features_df.Unemployment.median()))

mean_umempl_usa_201011 = (8.9 + 8.1) / 2

pd.DataFrame(features_df.Unemployment).boxplot(figsize=(20,1), vert=False)
plt.axvline(x=mean_umempl_usa_201011, color='red')
plt.show()

Wir haben die Daten des Datensatzes mit der durchschnittlichen Arbeitslosenquote der Vereinigten Staaten aus den Jahren 2011 und 2012 (Quelle: [Wikipedia](https://de.wikipedia.org/wiki/Arbeitsmarktstatistik_der_Vereinigten_Staaten#2011_bis_2018)) in Höhe von ```{{mean_umempl_usa_201011}}``` verglichen.

Dabei zeigt sich, dass die Daten in Walmarts erhoben wurden, die potentiell eher in Regionen mit niedrigerer Arbeitslosenquote angesiedelt sind.  
Dieses zeigt sich auch in den Ausreißern: während in Q1 und Q2 die Daten eher dicht gepackt sind, streut Q3 und vor allem Q4 stark.

__IsHoliday__

In [ ]:
pd.DataFrame(features_df.IsHoliday).apply(pd.value_counts).plot.pie(figsize=(5,5), y='IsHoliday', autopct='%.2f')
plt.show()

Wie in der Grafik zu sehen, sind in ```7.14%``` der betrachteten Datensätze Feiertage zu verzeichnen.  
In diesem Zeitraum liegen unter anderem:
+ Super Bowl
+ Labor Day
+ Thanksgiving
+ Black Friday
+ Weihnachten

#### Stores - Datensatz

Der Datensatz ```stores.csv``` enthält die Attribute ```Store```, ```Type``` und ```Size```.

__Store__

```Store``` stellt für die Datei den Primärschlüssel, dabei sind alle Stores von ID ```{{stores_df.Store.min()}}``` bis ID ```{{stores_df.Store.max()}}``` enthalten.

__Type__

Das Attribut ```Type``` enthält die Werte ```{{stores_df.Type.unique()[0]}}```, ```{{stores_df.Type.unique()[1]}}``` und ```{{stores_df.Type.unique()[2]}}```.  
Es gibt folgende unterschiedliche Typen von Märkten (Quelle: [Walmart Store Locator](https://corporate.walmart.com/our-story/our-business)), wobei wir uns nicht sicher sind, inwieweit dies gemapped werden kann:
+ Walmart Supercenter
+ Walmart Discount Store
+ Walmart Neighbourhood Market

Die Verteilung der Store-Typen gestaltet sich wie folgt:

In [ ]:
stores_df.groupby('Type')['Type'].count().plot(kind='bar', rot=0)
plt.ylabel("Anzahl")
plt.show()

__Size__

Nachdem die Daten anonymisiert sind, kann keine definitive Aussage darüber getroffen werden, in welcher Einheit die gegebenen Werte im Bereich von ```{{stores_df.Size.min()}}``` bis ```{{stores_df.Size.max()}}``` vorliegen.  
Allerdings entsprechen die Durchschnittswerte für die einzelnen Typen ziemlich exakt den Typen von Walmarts, die im letzten Punkt angesprochen worden waren:

<img src="./media/walmart_types.jpg" width="800"/>

In [ ]:
size_a_type = 182000
size_b_type = 106000
size_c_type = 38000

stores_df.boxplot(column='Size', by='Type', vert=False, figsize=(20,5))
plt.axvline(x=size_a_type, color='red')
plt.axvline(x=size_b_type, color='red')
plt.axvline(x=size_c_type, color='red')

plt.suptitle('')
plt.title('')
plt.xlabel('Size')
plt.ylabel('Type')
plt.show()

#### Erste Hypothesen
Um erste Hypothesen aufstellen zu können, die über unsere bisherigen Betrachtungen (beispielsweise zur Arbeitslosenquote) hinausgehen, müssen wir die Datensätze miteinander verbinden. Hierfür verweisen wir auf die Phase der __Data Preparation__.

### Betrachtung der Datenqualität
Die mangelnde Qualität der Features ```MarkDown 1-5``` wurde bereits angesprochen.  
Darüber hinausgehende Aussagen können erst in der Phase der __Data Preparation__ getroffen werden.

Auf den ersten Blick sieht es so aus, als würden die zur Verfügung stehenden Daten ausreichen, um erste Modelle zu bauen. Sie enthalten vorerst alle wichtigen Informationen. Falls die Qualität der Modelle nicht zufriedenstellend ist, ist die Struktur geeignet, um durch weitere Daten erweitert zu werden. 

## DATA PREPARATION: Neural Net Approach

Im Folgenden bereiten wir die soeben untersuchten Daten für das Modellieren vor.  
Dabei wird zuerst ein __zeitlicher Zuschnitt__ erfolgen, an den sich ein __Feature Engineering__ anschließt.  
Nach einer __Normalisierung__ sollen die Daten in 3D-Numpy-Arrays aufgebaut werden, um ein Training mit __Moving Window__ in __neuronalen Netzen__ zu ermöglichen.

### Zeitlicher Zuschnitt der Daten auf das Projektziel

Wie bereits erwähnt, ist es unser Ziel, aus den Daten von 2010 und 2011 das Weihnachtsgeschäft des Jahres 2011 zu prognostizieren. Daher fokussieren wir uns auf diesen Zeitraum:

In [ ]:
data_df.Date = pd.to_datetime(data_df.Date)

In [ ]:
xmas_data_df = data_df[((data_df.Date >= '2010-02-05') & (data_df.Date <= '2011-12-31'))]
xmas_data_df.reset_index(inplace=True, drop=True)

In [ ]:
display(xmas_data_df.loc[(xmas_data_df['Store'] == 1) & (xmas_data_df['Dept'] ==1)].head(2))
display(xmas_data_df.loc[(xmas_data_df['Store'] == 1) & (xmas_data_df['Dept'] ==1)].tail(2))

Wir betrachten also einen Zeitraum von ```{{xmas_data_df.Date.nunique()}}``` Wochen.

Fraglich ist, ob die Daten von zwei Jahren hier schon ausreichend einem Muster folgen - speziell, weil wir direkt die Ergebnisse des zweiten Weihnachtsgeschäfts prognostizieren wollen.  
Dazu visualisieren wir die beiden Jahre und betrachten vor allem die Wochen vor Weihnachten:

In [ ]:
xmas_data_df.groupby('Date').agg(['sum']).reset_index().plot(x='Date', y='Weekly_Sales',kind="bar", figsize=(20,7)).set_ylabel("Weekly Sales in 10 mio.")
plt.axhline(y=xmas_data_df['Weekly_Sales'][xmas_data_df.Date == '2010-11-26'].agg(['sum'])[0], color='black')
plt.axhline(y=xmas_data_df['Weekly_Sales'][xmas_data_df.Date == '2010-12-03'].agg(['sum'])[0], color='grey')
plt.axhline(y=xmas_data_df['Weekly_Sales'][xmas_data_df.Date == '2010-12-10'].agg(['sum'])[0], color='green')
plt.axhline(y=xmas_data_df['Weekly_Sales'][xmas_data_df.Date == '2010-12-17'].agg(['sum'])[0], color='orange')
plt.axhline(y=xmas_data_df['Weekly_Sales'][xmas_data_df.Date == '2010-12-24'].agg(['sum'])[0], color='red')
# Anpassung y-Achse über min- und max-Werte mit Faktor 0.1
plt.ylim(xmas_data_df.groupby('Date').agg(['sum']).Weekly_Sales.min()[0] * 0.9, xmas_data_df.groupby('Date').agg(['sum']).Weekly_Sales.max()[0] * 1.1)
plt.show()

In dieser Grafik lässt sich ein Muster des Verlaufs der Verkaufszahlen in der Weihnachstzeit erkennen, weshalb wir erhoffen, gute Prognosen aus den Daten ableiten zu können.

### Feature Engineering

#### Warnung: nächste Woche ist ein wichtiger Feiertag

Aufgrund der Bedeutung der Feiertage für die Verkaufszahlen, erweitern wir den Datensatz um da Feature ```next_week```, das folgende Werte annehmen kann:
+ ```black_friday```
+ ```xmas_1st_week```
+ ```xmas_2nd_week```
+ ```xmas_3rd_week```
+ ```xmas_4th_week```
+ ```new_year```

Die Hoffnung ist, dass die Algorithmen aus diesem Hinweis für die nächste Woche sinnvolle Schlüsse ziehen können.

In [ ]:
xmas_data_df.loc[xmas_data_df.Date == '2010-11-19', 'next_week'] = 'black_friday'
xmas_data_df.loc[xmas_data_df.Date == '2010-11-26', 'next_week'] = 'xmas_1st_week'
xmas_data_df.loc[xmas_data_df.Date == '2010-12-03', 'next_week'] = 'xmas_2nd_week'
xmas_data_df.loc[xmas_data_df.Date == '2010-12-10', 'next_week'] = 'xmas_3rd_week'
xmas_data_df.loc[xmas_data_df.Date == '2010-12-17', 'next_week'] = 'xmas_4th_week'
xmas_data_df.loc[xmas_data_df.Date == '2010-12-24', 'next_week'] = 'new_year'

xmas_data_df.loc[xmas_data_df.Date == '2011-11-18', 'next_week'] = 'black_friday'
xmas_data_df.loc[xmas_data_df.Date == '2011-11-25', 'next_week'] = 'xmas_1st_week'
xmas_data_df.loc[xmas_data_df.Date == '2011-12-02', 'next_week'] = 'xmas_2nd_week'
xmas_data_df.loc[xmas_data_df.Date == '2011-12-09', 'next_week'] = 'xmas_3rd_week'
xmas_data_df.loc[xmas_data_df.Date == '2011-12-16', 'next_week'] = 'xmas_4th_week'
xmas_data_df.loc[xmas_data_df.Date == '2011-12-23', 'next_week'] = 'new_year'

#### Wie lange dauerts noch bis Weihnachten?

Zusätzlich wäre es interessant zu erfahren, wie viele Tage es noch bis Weihnachten sind.  
Dazu dient das Feature ```days_2_xmas```:

In [ ]:
xmas_data_df['days_2_xmas'] = xmas_data_df.Date.apply(lambda x: (date(x.year,12,25) - x.date()).days)
xmas_data_df[['Date', 'days_2_xmas']].head()

### Umgang mit kategorialen Daten

Sowohl das soeben erstellte Attribut ```next_week```, als auch die Attribute ```Store_Type``` und ```IsHoliday``` sollten betrachtet werden, da die von uns eingesetzten neuronalen Netze nicht mit kategorialen Daten umgehen können.

#### next_week

Hier können wir praktischerweise die von ```pandas``` mitgelieferte Funktion ```get_dummies``` nutzen:

In [ ]:
xmas_data_df = pd.get_dummies(data=xmas_data_df, columns=['next_week'], prefix='nw')

In [ ]:
xmas_data_df[(xmas_data_df.Store == 1) & (xmas_data_df.Dept == 1) & (xmas_data_df.Date >= '2010-11-12') & (xmas_data_df.Date < '2010-12-31')]

#### Store_Type

In [ ]:
if 'Type' in stores_df.columns:
    stores_df = pd.get_dummies(data=stores_df, columns=['Type'], prefix='type')
stores_df.head()

#### IsHoliday

Eine gesonderte Behandlung für ```IsHoliday``` nicht eventuell nötig, weil ```boolean```sich in ```int``` casten lässt.  
Wir gehen allerdings auf Nummer sicher und nehmen den Netzen diese Arbeit ab:

In [ ]:
features_df.IsHoliday = features_df.IsHoliday.astype(int)
features_df.head()

### Merge Datasets

Da das Feature ```IsHoliday``` neben dem soeben umgewandelten ```IsHoliday``` noch redundant im File ```train.csv``` vorhanden ist, wird dieses hier gedroppt:

In [ ]:
features_df.Date = pd.to_datetime(features_df.Date)
if 'IsHoliday' in xmas_data_df.columns:
    xmas_data_df.drop(columns=['IsHoliday'],inplace=True)

Nachdem wir die Daten so erstellt haben, wie wir sie für den ersten Versuch nutzen wollen, können die Datensets ```features_df```, ```stores_df``` und ```xmas_data_df``` gemerged werden.

In [ ]:
merged_df = pd.merge(xmas_data_df,features_df,on=['Store','Date'])
merged_all_df = pd.merge(merged_df,stores_df,on='Store')
merged_all_df.head()

### Clean Data

Wie man in der Ausgabe oben sehen kann, enthält der Datenbestand noch __```NaN```-Werte__. Nachdem ein geeigneter Umgang mit diesen gefunden wurde, sollen noch die __Korrelationen__ betrachtet werden, um eventuell vorhandene und quasi redundante Features zu entfernen.

#### Umgang mit NaN

Zuerst können direkt wie bereits erwähnt die Attribute ```MarkDown 1-5 ``` entfernt werden:

In [ ]:
if 'MarkDown1' in merged_all_df.columns:
    merged_all_df.drop(columns=['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'],inplace=True)
merged_all_df.head()

Anschließend wird noch geklärt, inwieweit weitere ```NaN```-Werte vorhanden sind:

In [ ]:
merged_all_df.isnull().sum()

Hier scheinen wir Glück gehabt zu haben, weitere Aktionen wie Mitteln ect von Werten sind nicht nötig.

#### Korrelationen

Im Folgenden plotten wir uns die Korrelationsmatrix für unsere Features.  
Der Code ist der [Seaborn Doku](https://seaborn.pydata.org/examples/many_pairwise_correlations.html) entnommen und leicht angepasst.

In [ ]:
corr = merged_all_df.iloc[:,3:].corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(20, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, center=0, square=False, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()

Man kann direkt erkennen, dass es eine hohe Korrelation der Features ```Temperature_F``` und ```Temperature_C``` gibt - was verständlich ist, da wir ```Temperature_F``` selbst umgerechnet haben. Wir löschen ```Temperature_F```.

In [ ]:
if 'Temperature_F' in merged_all_df.columns:
    merged_all_df.drop(columns='Temperature_F',inplace=True)

Es freut uns natürlich, dass es eine Korrelation zwischen ```nw_new_year``` und ```Weekly_Sales``` gibt - diese Feature könnte also genutzt werden, um dieses einmalige Ereignis wiederzuerkennen.  

Zwischen ```nw_xmas_1st_week``` und ```IsHoliday``` besteht ein Zusammenhang, da in dieser Woche immer der _Black Friday_ liegt.

Spannend ist auch, dass es anscheinend einen Zusammenhang zwischen der ```Temperatur_C``` und ```Fuel_Price```, ```Unemployment``` und ```CPI``` gibt - während ```CPI```und ```Fuel_Price``` tatsächlich eine negative Korrelation aufweisen und ```Unemployment``` und ```Fuel_Price``` neutral zueinander stehen.  
Dass ```CPI``` und ```Unemployment``` wiederum negativ korrelieren ergibt Sinn.

Auch, dass es anscheinend an den interessanten Feiertagen eher kälter ist, überrascht nicht.

Auch die Zusammenhänge zwischen (Store-)```Size```, (Store-)```Type``` und absoluten ```Weekly_Sales``` überrascht nicht.

__Fazit:__
Wir sind am Ende der Datenbereinigung angekommen und haben damit einen Datensatz mit ```{{merged_all_df.shape[0]}}``` Messpunkten und aktuell ```{{merged_all_df.shape[1]}}``` Attibuten.

### Normalize Data

An dieser Stelle würde im Normalfall eine Normalisierung der Daten anstehen, da die Skalen wie bereits gesehen sehr unterschiedlich ausfallen, nicht nur zwischen den ehemals kategorialen und den stetigen Attributen.  
Um zu ermöglichen, dass die Daten in Trainings- und Testdaten gesplittet werden können und die Normalisierung nur anhand der Trainingsdaten stattfinden kann, findet die Normalisierung in der Splitting-Funktion statt.

### Dictionary bauen: Neural Network Approach

Um die einzelnen im Folgenden erzeugten Dataframes, Arrays, Modelle ect sinnvoll im Speicher vorhalten zu können, arbeiten wir mit einem Dictionary, das in drei Ebenen angelegt ist:  
__dict [ Store-Department-Identifier ] [ einzelne Dateframes, Arrays, ect und Modeltyp [ Modell, Auswertung ect ] ]__

Die Idee haben wir von [Kevin Palmer](https://github.com/kevinpalm/walmart_sales_forecast/blob/master/final_model.py) geklaut, die Umsetzung erfolgte allerdings auf sehr unterschiedliche Weise.

In [ ]:
multi_dict = {}

Die weiteren Arbeiten sollen immer auf Basis einer Store-Department-Kombination erfolgen (zb für ```Store 1``` und ```Department 1```).  
Dies bedeutet, dass pro solcher Kombination Modelle gebaut und bewertet werden sollen.  
Dazu generieren wir Identifier:

In [ ]:
merged_all_df['Store_Dept'] = merged_all_df.Store.astype(str) + '_' + merged_all_df.Dept.astype(str)

Wir haben es insgesamt mit ```{{merged_all_df.Store_Dept.nunique()}}``` Kombinationen zu tun.

Im Folgenden werden diese Identifier genutzt, um die erste Ebene des Dictionaries anzulegen.  
Dabei ist zu beachten, dass der folgende Code nicht standardmäßig ausgeführt wird, um die weitere Ausführung des Notebooks nicht unverhältnismäßig zu verlangsamen.  
Im Bedarfsfall sollte die nächste Zelle aktiviert und die übernächste Zelle ("Abgespeckte Version") deaktiviert werden:

Damit die Prüfungen im weitern Verlauf alle funktionieren, __MUSS__ die Store-Department-Kombination ```1_1``` __IMMER__ enthalten sein!

In [ ]:
store_dept_subgroup = ['1_1', '2_1']

for store_dept in store_dept_subgroup:
    multi_dict[store_dept] = {}

### Split Datasets

In diesem Unterkapitel nehmen wir die Aufspaltung nach __Trainings-, Test- und Validationsdaten__ vor.

#### Validation je Store und Depatment

<img src="./media/calendar.jpg" width="500"/>

Das Ziel ist es, aus den Daten der schwarz markierten Wochen die Verkaufszahlen der grün markierten Woche zu ermitteln. Die Daten sind dabei jeweils am Freitag für die vergangene Woche erfasst.  
Die rot markierte Woche ist überflüssig und kann gelöscht werden:

In [ ]:
date_cutoff = date(2011,12,30)
merged_all_df.drop(merged_all_df[(merged_all_df.Date >= date_cutoff)].index, inplace=True)

Damit ist das jüngste Datum im Datenbestand der ```{{merged_all_df.Date.max()}}```.

__Anmerkung:__  
Da Bäume einen anderen Aufbau der Daten benötigen, kopieren wir den aktuellen Stand des Dataframes, da wir weitere Änderungen direkt auf dem Dataframe machen werden:

In [ ]:
merged_all_for_tree_df = merged_all_df.copy()

Nun können die oben markierten Validierungsdaten für alle Store-Department-Kombinationen ermittelt und in das Dictionary gespeichert werden:

In [ ]:
date_start_windowing = date(2011,11,25)
date_validation_week = date(2011,12,23)

if date_validation_week in merged_all_df.Date.dt.date.values:
    for store_dept in multi_dict.keys():
        multi_dict[store_dept]['val_data_df'] = merged_all_df[(merged_all_df.Date >= date_start_windowing)&(merged_all_df.Store_Dept == store_dept)].reset_index(drop=True)

Damit hat das Dictionary folgenden Inhalt:

In [ ]:
multi_dict['1_1'].keys()

In [ ]:
multi_dict['1_1']['val_data_df'][['Store', 'Dept', 'Date', 'Weekly_Sales']]

Um ein Leaken der Validierungsdaten ins Trainig zu verhindern, wird die entscheidende Woche aus dem Dataframe getilgt:

In [ ]:
merged_all_df.drop(merged_all_df[(merged_all_df.Date == date_validation_week)].index, inplace=True)

Das jüngste Datum ist damit der ```{{merged_all_df.Date.max()}}``` und die Validierungs-Woche ist nicht mehr enthalten:

In [ ]:
merged_all_df[(merged_all_df.Date >= date_start_windowing)&(merged_all_df.Store_Dept == '1_1')][['Store', 'Dept', 'Date', 'Weekly_Sales']].reset_index(drop=True)

#### Train-Test-Daten je Store und Department

Nachdem die Validierungsdaten sauber abgespeichert sind, kümmern wir uns um die Trainings- und Testdaten.  
Dabei nehmen wir auch eine Restrukturierung vor, wobei vor allem das Attribut ```Weekly_Sales``` an den Schluss geschoben wird:

In [ ]:
def restructure_df_for_target_variable(df):
    return df[['Date',
       'nw_black_friday', 'nw_new_year', 'nw_xmas_1st_week', 'nw_xmas_2nd_week',
       'nw_xmas_3rd_week', 'nw_xmas_4th_week', 'days_2_xmas', 'Fuel_Price',
       'CPI', 'Unemployment', 'IsHoliday', 'Temperature_C', 'Size', 'type_A',
       'type_B', 'type_C', 'Weekly_Sales']].reset_index(drop=True)

In [ ]:
for store_dept in multi_dict.keys():
    multi_dict[store_dept]['train_test_df'] = restructure_df_for_target_variable(merged_all_df[(merged_all_df.Store_Dept == store_dept)])

Damit hat das Dictionary die folgende Form:

In [ ]:
multi_dict['1_1'].keys()

Und der Dataframe sieht wie folgt aus:

In [ ]:
multi_dict['1_1']['train_test_df'].head()

#### Train-Test-Split

Nachdem die Trainigs- und Testdaten isoliert wurden, kann jetzt der Split erfolgen.  
Im Rahmen des Splittes findet wie gesagt auch die __Normalisierung__ statt.  
Am besten man beginnt beim Studium folgender Funktionen mit der Funktion ```split_data()```:

In [ ]:
def scale_data(data_np, scaler=None):
    if not scaler:
        scaler = StandardScaler().fit(data_np)
    return scaler.transform(data_np), scaler

In [ ]:
def get_X_y_scalers(data_df):
    data_np = data_df.drop(columns='Date').values
    X_scaler = StandardScaler().fit(data_np[:,:-1])
    y_scaler = StandardScaler().fit(data_np[:,-1].reshape(-1, 1))
    return X_scaler, y_scaler

In [ ]:
def X_y_split(data_df, lookback, delay, X_y_scaler=None):
    data_np = data_df.drop(columns='Date').values
    data_rows = np.arange(lookback, data_np.shape[0] - delay)
    
    scaled_data_np, X_y_scaler = scale_data(data_np, X_y_scaler)

    X_np = np.zeros((len(data_rows), lookback, scaled_data_np.shape[1]-1))
    y_np = np.zeros((len(data_rows),))
    
    for i, row in enumerate(data_rows):
        indices = range(row - lookback, row)
        X_np[i] = scaled_data_np[indices,:-1]
        y_np[i] = scaled_data_np[row + delay][-1]
    
    return X_np, y_np, X_y_scaler

In [ ]:
def split_data(data, lookback, delay, cutoff_date):
    train_samples, train_targets, X_y_scaler = X_y_split(data[data.Date <= cutoff_date], lookback, delay)
    test_samples, test_targets, X_y_scaler = X_y_split(data[data.Date > cutoff_date], lookback, delay, X_y_scaler)
    return train_samples, train_targets, test_samples, test_targets, X_y_scaler

Nachdem die Funktionenn definiert wurden, werden die Parameter für den Split gesetzt.  
Wir wollen vier Wochen aus der Vergangenheit betrachten, um die aktuelle Woche zu prognostizieren, die Daten ab dem ```2011-05-01``` nutzen wir zum Testen:

In [ ]:
lookback = 4
delay = 0
cutoff_date = '2011-05-01'

Im ersten Schritt speichern wir die beiden Scaler für ```X_scaler``` und ```y_scaler```, die wir später für den Sanity Check benötigen:

In [ ]:
for store_dept in multi_dict.keys():
    df = multi_dict[store_dept]['train_test_df']
    X_scaler, y_scaler = get_X_y_scalers(df[df.Date <= cutoff_date])
    multi_dict[store_dept]['X_scaler'] = X_scaler
    multi_dict[store_dept]['y_scaler'] = y_scaler

In [ ]:
multi_dict['1_1'].keys()

Im zweiten Schritt nehmen wir den Train-Test-X-y-Split vor.  
Dabei speichern wir zusätzlich den ```X-y-Scaler``` ab, den wir später für den Validierungsdatensatz benötigen:

In [ ]:
for store_dept in multi_dict.keys():
    X_train, y_train, X_test, y_test, X_y_scaler = split_data(multi_dict[store_dept]['train_test_df'], lookback, delay, cutoff_date)
    multi_dict[store_dept]['X_train'] = X_train
    multi_dict[store_dept]['y_train'] = y_train
    multi_dict[store_dept]['X_test'] = X_test
    multi_dict[store_dept]['y_test'] = y_test
    multi_dict[store_dept]['X_y_scaler'] = X_y_scaler

In [ ]:
multi_dict['1_1'].keys()

__Sanity Check:__
Nach den nicht ganz unkomplexen Vorgängen wollen wir wissen, ob das Ergebnis unseren Erwartungen entspricht:

Der ```X_train```-Datensatz sollte ein 3D-Numpy-Array sein, das ```61 Iterationen``` von ```4-Wochen-Vorschauen``` mit ```je 16 Attributen``` enthält:

In [ ]:
multi_dict['1_1']['X_train'].shape

Das passt.  
Jetzt lesen wir die erste Woche der Iteration aus:

In [ ]:
multi_dict['1_1']['X_train'][0][0]

Nachdem uns die normalisierten Werte nicht viel sagen, rechnen wir diese mit dem ```X_scaler``` zurück:

In [ ]:
multi_dict['1_1']['X_scaler'].inverse_transform(multi_dict['1_1']['X_train'][0][0])

Und vergleichen diese mit der ersten Woche im unbehandelten Dataframe:

In [ ]:
multi_dict['1_1']['train_test_df'].iloc[0]

> __Fazit:__ Das sieht soweit ganz gut aus!

Jetzt bleibt noch, ```y_train``` für die erste Woche auf die selbe Art zu prüfen:

In [ ]:
multi_dict['1_1']['y_train'][0]

In [ ]:
multi_dict['1_1']['y_scaler'].inverse_transform(multi_dict['1_1']['y_train'][0].reshape(-1, 1))

In [ ]:
multi_dict['1_1']['train_test_df'].Weekly_Sales.iloc[lookback + delay]

In [ ]:
multi_dict['1_1']['train_test_df'].iloc[4].Weekly_Sales

> __Fazit:__ Auch das passt.

#### Validation to np

Auch die Validations-Daten sollten als np-Array vorliegen, um später Auswertungen machen zu können:

In [ ]:
for store_dept in multi_dict.keys():
    multi_dict[store_dept]['val_data_df'] = restructure_df_for_target_variable(multi_dict[store_dept]['val_data_df'])

In [ ]:
for store_dept in multi_dict.keys():
    X_np, y_np, foo = X_y_split(multi_dict[store_dept]['val_data_df'], lookback, delay, multi_dict[store_dept]['X_y_scaler'])
    multi_dict[store_dept]['X_val'] = X_np
    multi_dict[store_dept]['y_val'] = y_np
multi_dict['1_1'].keys()

#### Set prediction goal

Um zuverlässig auf das Ziel der Validations-Daten zugreifen zu können, wir auch dieses abgespeichert:

In [ ]:
for store_dept in multi_dict.keys():
    multi_dict[store_dept]['prediction_goal'] = multi_dict[store_dept]['val_data_df'].iloc[-1].Weekly_Sales

In [ ]:
multi_dict['1_1'].keys()

In [ ]:
multi_dict['1_1']['prediction_goal']

## MODELING: Neural Network Approach

Nach den raumgreifenden Erläuterungen zur Data Preparation erscheint das folgende Kapitel fast karg.  
Zuerst betrachten wir eine einfache __Baseline__.  
Dann __trainieren__ wir verschiedene Algorithmen mit den erstellten Daten.  
Im Anschluss wagen wir eine __kurze Evaluation__ der ersten Ergebnisse.

### Baseline

Die Baseline folgt dem einfachen Prinzip, dass die ```Weekly_Sales``` der aktuellen Woche als Prognose für die kommende Woche angenommen werden. Dies mag unter dem Jahr eine gar nicht so schlechte Methode sein (im Vergleich zum Input), für das Weihnachtsgeschäft kann sie allerdings nur versagen.
Der Einfachheit halber erfolgt die Erstellung nur für Store-Department-Kombination ```1_1```.

In [ ]:
def get_baseline_prediction_values(df):
    prognose_sales = df.iloc[-2].Weekly_Sales
    actual_sales = df.iloc[-1].Weekly_Sales
    return [prognose_sales, actual_sales, actual_sales - prognose_sales]

```Store 1``` mit ```Department 1```:

In [ ]:
baseline_prediction_values = get_baseline_prediction_values(multi_dict['1_1']['val_data_df'])
print("Baseline Prognose:\t{}".format(baseline_prediction_values[0]))
print("Tatsächliche Sales:\t{}".format(multi_dict['1_1']['prediction_goal']))
print("Absoluter Fehler:\t{}".format(baseline_prediction_values[2]))

So schlecht ist die Baseline-Prognose für ```1_1``` nicht. Können wir es besser?

Prognose für ```Store 2``` mit ```Department 1```:

In [ ]:
baseline_prediction_values = get_baseline_prediction_values(multi_dict['2_1']['val_data_df'])
print("Baseline Prognose:\t{}".format(baseline_prediction_values[0]))
print("Tatsächliche Sales:\t{}".format(multi_dict['2_1']['prediction_goal']))
print("Absoluter Fehler:\t{}".format(baseline_prediction_values[2]))

Hier ist der Fehler schon bedeutend heftiger.

### Training der neuronale Netze

Im Folgenden wird mit verschiedenen Typen von neuronalen Netzen trainiert.  
Die Ergebnisse werden jeweils als neues Dictionary in das bekannte Dictionary integriert, um für die spätere Auswertung zur Verfügung zu stehen.

#### 2 Layer Perceptron

Auch wenn ein MLP kein RNN ist, soll es hier als Vergleichswert dienen.

In [ ]:
for store_dept in multi_dict.keys():
    model = Sequential()
    model.add(layers.Flatten(input_shape=(lookback, multi_dict[store_dept]['X_train'].shape[-1])))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1))
    print(model.summary())
    model.compile(optimizer=RMSprop(), loss='mae')
    history = model.fit(multi_dict[store_dept]['X_train'],
                        multi_dict[store_dept]['y_train'],
                        epochs=20,
                        validation_data=(multi_dict[store_dept]['X_test'], multi_dict[store_dept]['y_test']))
    multi_dict[store_dept]['2MLP'] = {}
    multi_dict[store_dept]['2MLP']['model'] = model
    multi_dict[store_dept]['2MLP']['history'] = history
    multi_dict[store_dept]['2MLP']['score'] = model.evaluate(multi_dict[store_dept]['X_val'], multi_dict[store_dept]['y_val'])
    multi_dict[store_dept]['2MLP']['prediction'] = multi_dict[store_dept]['y_scaler'].inverse_transform(model.predict(multi_dict[store_dept]['X_val']).reshape(-1, 1))[0][0]

#### Simple GRU Model

In [ ]:
for store_dept in multi_dict.keys():
    model = Sequential()
    model.add(layers.GRU(32, input_shape=(None, multi_dict[store_dept]['X_train'].shape[-1])))
    model.add(layers.Dense(1))
    print(model.summary())
    model.compile(optimizer=RMSprop(), loss='mae')
    history = model.fit(multi_dict[store_dept]['X_train'],
                        multi_dict[store_dept]['y_train'],
                        epochs=20,
                        validation_data=(multi_dict[store_dept]['X_test'], multi_dict[store_dept]['y_test']))
    multi_dict[store_dept]['GRU'] = {}
    multi_dict[store_dept]['GRU']['model'] = model
    multi_dict[store_dept]['GRU']['history'] = history
    multi_dict[store_dept]['GRU']['score'] = model.evaluate(multi_dict[store_dept]['X_val'], multi_dict[store_dept]['y_val'])
    multi_dict[store_dept]['GRU']['prediction'] = multi_dict[store_dept]['y_scaler'].inverse_transform(model.predict(multi_dict[store_dept]['X_val']).reshape(-1, 1))[0][0]

#### GRU with Dropout

Einen Dropout einzuführen sieht auf den ersten Blick sinnlos aus, wenn man bedenkt, dass uns nur 100 Datensätze zur Verfügung stehen. Inwieweit diese Befürchtungen gerechtfertigt sind, wird man in der Evaluation sehen.

In [ ]:
for store_dept in multi_dict.keys():
    model = Sequential()
    model.add(layers.GRU(32, dropout=0.2, recurrent_dropout=0.2, input_shape=(None, multi_dict[store_dept]['X_train'].shape[-1])))
    model.add(layers.Dense(1))
    print(model.summary())
    model.compile(optimizer=RMSprop(), loss='mae')
    history = model.fit(multi_dict[store_dept]['X_train'],
                        multi_dict[store_dept]['y_train'],
                        epochs=20,
                        validation_data=(multi_dict[store_dept]['X_test'], multi_dict[store_dept]['y_test']))
    multi_dict[store_dept]['GRU_drop'] = {}
    multi_dict[store_dept]['GRU_drop']['model'] = model
    multi_dict[store_dept]['GRU_drop']['history'] = history
    multi_dict[store_dept]['GRU_drop']['score'] = model.evaluate(multi_dict[store_dept]['X_val'], multi_dict[store_dept]['y_val'])
    multi_dict[store_dept]['GRU_drop']['prediction'] = multi_dict[store_dept]['y_scaler'].inverse_transform(model.predict(multi_dict[store_dept]['X_val']).reshape(-1, 1))[0][0]

#### Stacked GRU

In [ ]:
for store_dept in multi_dict.keys():
    model = Sequential()
    model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5, return_sequences=True, input_shape=(None, multi_dict[store_dept]['X_train'].shape[-1])))
    model.add(layers.GRU(64, activation='relu', dropout=0.1, recurrent_dropout=0.5))
    model.add(layers.Dense(1))
    print(model.summary())
    model.compile(optimizer=RMSprop(), loss='mae')
    history = model.fit(multi_dict[store_dept]['X_train'],
                        multi_dict[store_dept]['y_train'],
                        epochs=20,
                        validation_data=(multi_dict[store_dept]['X_test'], multi_dict[store_dept]['y_test']))
    multi_dict[store_dept]['GRU_stack'] = {}
    multi_dict[store_dept]['GRU_stack']['model'] = model
    multi_dict[store_dept]['GRU_stack']['history'] = history
    multi_dict[store_dept]['GRU_stack']['score'] = model.evaluate(multi_dict[store_dept]['X_val'], multi_dict[store_dept]['y_val'])
    multi_dict[store_dept]['GRU_stack']['prediction'] = multi_dict[store_dept]['y_scaler'].inverse_transform(model.predict(multi_dict[store_dept]['X_val']).reshape(-1, 1))[0][0]

### Evaluation der Modelle

Im Folgenden sollen die Ergebnisse der Modelle geplottet werden.

In [ ]:
def plot_model_results(model_results, prediction_goal):  
    loss = model_results['history'].history['loss']
    val_loss = model_results['history'].history['val_loss']
    min_val_loss=np.min(val_loss)
    epochs = range(1, len(loss) + 1)
    plt.figure()
    plt.plot(epochs[1:], loss[1:], 'bo', label='Training loss')
    plt.plot(epochs[1:], val_loss[1:], 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()
    print('Minimun loss on validation data:\t{}'.format(min_val_loss))
    print('Model score:\t\t\t\t{}'.format(model_results['score']))
    print('Prediction Goal:\t\t\t{}'.format(prediction_goal))
    print('Prediction result:\t\t\t{}'.format(model_results['prediction']))
    print('Absolute error:\t\t\t\t{}'.format(model_results['prediction'] - prediction_goal))

#### 2 Layer Perceptron

In [ ]:
plot_model_results(multi_dict['1_1']['2MLP'], multi_dict['1_1']['prediction_goal'])

In [ ]:
plot_model_results(multi_dict['2_1']['2MLP'], multi_dict['2_1']['prediction_goal'])

#### Simple GRU

In [ ]:
plot_model_results(multi_dict['1_1']['GRU'], multi_dict['1_1']['prediction_goal'])

In [ ]:
plot_model_results(multi_dict['2_1']['GRU'], multi_dict['2_1']['prediction_goal'])

#### GRU with Dropout

In [ ]:
plot_model_results(multi_dict['1_1']['GRU_drop'], multi_dict['1_1']['prediction_goal'])

In [ ]:
plot_model_results(multi_dict['2_1']['GRU_drop'], multi_dict['2_1']['prediction_goal'])

#### Stacked GRU

In [ ]:
plot_model_results(multi_dict['1_1']['GRU_stack'], multi_dict['1_1']['prediction_goal'])

In [ ]:
plot_model_results(multi_dict['2_1']['GRU_stack'], multi_dict['2_1']['prediction_goal'])

#### Beurteilung und Schlussfolgerungen

+ Die Modelle sehen leider bei jedem Durchlauf mit identischen Einstellungen komplett unterschiedlich aus.
    + Wenn ein Modell einmal ein gutes Ergebnis liefert, dann nur durch Zufall.
+ Eine Änderung der Epochen hat keinerlei sinnvolle Auswirkungen.
+ Verschiedene Modelle lassen sich nicht miteinander vergleichen.
+ Wir gehen davon aus, dass wir mit genau 100 Samples für Training und Test zu wenig Daten für ein sinnvolles Training zur Verfügung haben.

Ein Rückschritt zur Data Preparation scheint angebracht zu sein, um eventuell mithilfe von Regression mit Bäumen bessere Ergebnisse zu erzielen.

## DATA PREPARATION: Tree Approach

### Dictionary bauen

In [ ]:
tree_dict = {}

Um im Anschluss an die Modellierung mithilfe von Regressionen die Ergebnisse vergleichen zu können, benötigen wir ein neues Dictionary dafür.

Die Schlüssel für die erste Ebene anlegen  
__HIER IST DIE WEICHE FÜR STORE_DEPT-Kombos__

Damit die Prüfungen im weiteren Verlauf alle funktionieren, __MUSS__ die Store-Department-Kombination ```1_1``` __IMMER__ enthalten sein!

In [ ]:
store_dept_subgroup = ['1_1']

for store_dept in store_dept_subgroup:
    tree_dict[store_dept] = {}
print(len(tree_dict))

### Split Datasets

#### Validation je Store und Department

<img src="./media/calendar_tree.jpg" width="500"/>

Auch bei den Bäumen kann die letzte Woche aus den Daten entfernt werden. Somit ist hier ebenfalls der ```{{merged_all_for_tree_df.Date.max()}}``` das jüngste Datum im Datenbestand.  
  
Hier benutzen wir die Woche vom ```10.12.2011``` bis zum ```16.12.2011``` zum Vorhersagen der Weihnachtswoche. Durch das Shiften der Weekly_Sales befindet sich der Wert des ```23.12.2011``` in diesem Eintrag.

In [ ]:
for store_dept in tree_dict.keys():
    tmp = merged_all_for_tree_df[merged_all_for_tree_df.Store_Dept == store_dept].iloc[-2:]
    tmp.Weekly_Sales = tmp.Weekly_Sales.shift(periods=-1)
    tree_dict[store_dept]['val_data_df'] = tmp.iloc[0]
    
tree_dict['1_1']['val_data_df']

#### Train-Test-Daten je Store und Department

In [ ]:
for store_dept in tree_dict.keys():
    tmp = restructure_df_for_target_variable(merged_all_for_tree_df[(merged_all_for_tree_df.Store_Dept == store_dept)])
    tmp.Weekly_Sales = tmp.Weekly_Sales.shift(periods=-1)
    tree_dict[store_dept]['train_test_df'] = tmp.iloc[:-2]

Hier wird der Shift umgesetzt, dazu werden jeweils zwei Zeilen des Dataframes ausgegeben und die Weekly_Sales vom unteren zum oberen versetzt.

In [ ]:
tree_dict['1_1'].keys()

In [ ]:
tree_dict['1_1']['train_test_df'].tail()

#### Train-Test-Split

In [ ]:
for store_dept in tree_dict.keys():
    X = tree_dict[store_dept]['train_test_df'].iloc[:,:-1]
    y = tree_dict[store_dept]['train_test_df'].iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=1,shuffle=False)
    X_scaler, y_scaler = get_X_y_scalers(tree_dict[store_dept]['train_test_df'])
    X_train_np, foo = scale_data(X_train.drop(columns='Date').values, X_scaler)
    y_train_np, foo = scale_data(y_train.values.reshape(-1, 1), y_scaler)
    X_test_np, foo = scale_data(X_test.drop(columns='Date').values, X_scaler)
    y_test_np, foo = scale_data(y_test.values.reshape(-1, 1), y_scaler)
    foo, X_y_scaler = scale_data(tree_dict[store_dept]['train_test_df'].drop(columns='Date').values)
    tree_dict[store_dept]['X_train'] = X_train_np
    tree_dict[store_dept]['y_train'] = y_train_np
    tree_dict[store_dept]['X_test'] = X_test_np
    tree_dict[store_dept]['X_test_with_dates'] = X_test
    tree_dict[store_dept]['y_test'] = y_test_np
    tree_dict[store_dept]['X_scaler'] = X_scaler
    tree_dict[store_dept]['y_scaler'] = y_scaler
    tree_dict[store_dept]['X_y_scaler'] = X_y_scaler

Da wir bei den Bäumen nur eine Woche zum predicten benutzen kann die Funktion ```train_test_split``` verwendet werden. Die aufgesplitteten Daten skalieren wir mit unserer Funktion aus Kapitel 3.8.3. Zusätzlich zu diesen Datensets speichern wir das nicht skalierte ```X_test``` Dataset in dem Dict ab, damit die Ergebnisse später auf die Datumswerte zurückgeführt werden können (bei ```shuffle=True``` besonders wichtig)

In [ ]:
tree_dict['1_1'].keys()

#### Validation to np

In [ ]:
for store_dept in tree_dict.keys():
    tmp = restructure_df_for_target_variable(pd.DataFrame(tree_dict[store_dept]['val_data_df']).transpose()).drop(columns='Date')
    tmp_np, foo = scale_data(tmp.values, X_y_scaler)
    tree_dict[store_dept]['X_val'] = tmp_np[:,:-1]
    tree_dict[store_dept]['y_val'] = tmp_np[:,-1]

In [ ]:
tree_dict['1_1'].keys()

#### Set prediction goal

In [ ]:
for store_dept in tree_dict.keys():
    tree_dict[store_dept]['prediction_goal'] = tree_dict[store_dept]['val_data_df'].Weekly_Sales

In [ ]:
tree_dict['1_1'].keys()

In [ ]:
tree_dict['1_1']['prediction_goal']

In [ ]:
tree_dict['1_1']['y_scaler'].inverse_transform(tree_dict['1_1']['y_val'])

## MODELING: Tree Approach

Im diesem Abschnitt wird GradientBoosting Regression verwendet, um ein Modell zu trainieren.

### Gradient Boosting Regression

In [ ]:
## .ravel() -> transforming column-vector in 1D-Array
gb_regressor = GradientBoostingRegressor(n_estimators=400, random_state=1)
gb_regressor.fit(tree_dict['1_1']['X_train'], tree_dict['1_1']['y_train'].ravel())
print(gb_regressor)

In [ ]:
tree_dict['1_1']['y_pred'] = gb_regressor.predict(tree_dict['1_1']['X_test'])

### Evaluation der Modelle

#### MAE

In [ ]:
gb_mae = mean_absolute_error(tree_dict['1_1']['y_test'],tree_dict['1_1']['y_pred'])
tree_dict['1_1']['y_scaler'].inverse_transform(np.array([gb_mae]))

#### Feature Importance

In [ ]:
df_feat_imp = pd.DataFrame(gb_regressor.feature_importances_).transpose()
df_feat_imp.columns = tree_dict['1_1']['train_test_df'].columns[1:-1]
df_feat_imp.transpose()
df_feat_imp.transpose().sort_values(0,ascending=False).plot(kind='bar',figsize=(15,5),rot=45,legend=False)
plt.show()

An der Feature Importance kann abgelesen werden, dass die ```days_2_xmas``` unser wichtigstes Attribut darstellt. Durch Veränderung des Datensplits kann die Reihenfolge verändert werden (z.B. Setzt man das Attribut ```schuffle=True``` ist die Temperatur das wichtigste Attribut).

#### Gegenüberstellung der Vorhersage und der Testdaten

In [ ]:
test_dates = tree_dict['1_1']['X_test_with_dates']['Date'].values
plot_y_pred = tree_dict['1_1']['y_scaler'].inverse_transform(tree_dict['1_1']['y_pred'])
plot_y_test = tree_dict['1_1']['y_scaler'].inverse_transform(tree_dict['1_1']['y_test'].ravel())

plot_df = pd.DataFrame([test_dates,plot_y_pred,plot_y_test]).transpose().rename(columns={0:'Date',1:'pred_nw',2:'test_nw'})
plot_df.Date = pd.to_datetime(plot_df.Date)
plot_df.sort_values('Date').plot(x='Date',kind='bar',figsize=(15,6))
plt.show()

**Achtung!:** Die Sales Werte bezeichnen den Sales für die kommende Woche, ausgehend von dem angezeigten Datum!

An dieser Grafik lässt sich ablesen, dass unsere prediction die Steigungen und Gefälle zwar berücksichtigt, aber leider der Wert nicht mit den Testdaten übereinstimmt.  
Um die Weihnachtszeit nähert sich dies allerdings ziemlich nah an, weshalb es eventuell möglich ist unser Ziel, die Weihnachtsverkäufe 2011, mit einem guten Ergebnis vorherzusagen.

#### Prediction with Validation Data

In [ ]:
tree_dict['1_1']['y_val_pred'] = gb_regressor.predict(tree_dict['1_1']['X_val'])

In [ ]:
gb_val_pred = tree_dict['1_1']['y_scaler'].inverse_transform(tree_dict['1_1']['y_val_pred'])
print(gb_val_pred)

#### Beurteilung und Schlussfolgerung

Die prediction für die Weihnachstzeit 2011 liefert ```{{gb_val_pred[0]}}``` als Ergebnis. Gegenüber unseres prediction goals in Höhe von ```{{tree_dict['1_1']['y_scaler'].inverse_transform(tree_dict['1_1']['y_val'])[0]}}``` ist dies leider kein gutes Ergebnis.  
Wenn beim Splitten der Daten ```shuffle=True``` gesetzt ist, werden Werte unter 30.000$ vorhergesagt.  
Ebenfalls getestet haben wir diese Vorhersage mit einem ```DecisionTreeRegressor``` und einem ```RandomForestRegressor```, welche jedoch beide schlechtere Ergebnisse lieferten.

Da die Feature Importance die Temperatur als sehr wichtiges Attribut ausgibt, müssten die Stores weiter unterteilt werden, da das Modell eventuell nur bei Stores in Regionen mit ***normalen*** Klimabedingungen gut funktioniert, aber in unserem Datenset höchstwahrscheinlich auch welche aus Alaska, Florida, etc. enthalten sind.

## EVALUATION: Gegenüberstellung NN und Tree

In [ ]:
mlp_pred = multi_dict['1_1']['2MLP']['prediction']
gru_pred = multi_dict['1_1']['GRU']['prediction']
gru_drop_pred = multi_dict['1_1']['GRU_drop']['prediction']
gru_stack_pred = multi_dict['1_1']['GRU_stack']['prediction']
y_val = tree_dict['1_1']['y_scaler'].inverse_transform(tree_dict['1_1']['y_val'])
mpl_mae = multi_dict['1_1']['y_scaler'].inverse_transform(np.array([np.min(multi_dict['1_1']['2MLP']['history'].history['val_loss'])]))
gru_mae = multi_dict['1_1']['y_scaler'].inverse_transform(np.array([np.min(multi_dict['1_1']['GRU']['history'].history['val_loss'])]))
gru_drop_mae = multi_dict['1_1']['y_scaler'].inverse_transform(np.array([np.min(multi_dict['1_1']['GRU_drop']['history'].history['val_loss'])]))
gru_stack_mae = multi_dict['1_1']['y_scaler'].inverse_transform(np.array([np.min(multi_dict['1_1']['GRU_stack']['history'].history['val_loss'])]))
gb_tree_mae = tree_dict['1_1']['y_scaler'].inverse_transform(np.array([gb_mae]))

In [ ]:
eval_df = pd.DataFrame([[mlp_pred,gru_pred,gru_drop_pred,gru_stack_pred,gb_val_pred[0],y_val[0]]])
eval_df = eval_df.append([[mpl_mae[0],gru_mae[0],gru_drop_mae[0],gru_stack_mae[0],gb_tree_mae[0]]]).reset_index(drop=True)
eval_df = eval_df.rename(index={0:'prediction',1:'MAE'},columns={0:'Prediction 2 Layer Perceptron',1:'Prediction GRU',2:'Prediction GRU Dropout',3:'Prediction Stacked GRU',4:'Prediction GBR',5:'Prediction Goal'})

In [ ]:
eval_df.plot(kind='bar',rot=0, figsize=(15,5))
plt.show()

Im aktuellen Durchlauf führt die Verwendung von ```GRU``` zum besten Ergebnis. Da dieses Ergebnis allerdings bei jedem Durchlauf stark schwanken, kann hier keine genaue Aussage zu getroffen werden.  
Bei allen 5 Modellierungsvarianten liegt der ```Mean absolute error``` ziemlich hoch, da ein Unterschied von 30.000$ als erheblich anzusehen ist.

## DEPLOYMENT / Lessons Learned

Im Folgenden haben wir uns Gedanken gemacht, welche Schritte wir sinnvoll fanden, und was wir beim nächsten Mal anders machen würden.

### Positiv

+ Durch den hohen Fokus auf die Phasen Data Understanding und Data Preparation haben wir gefühlt einen sehr guten Überblick über die Attribute des Datenbestandes und die inneren Zusammenhänge bekommen.
+ Das standardisierte Vorgehen mit dem Vorhalten aller In- und Outputs über ein geschachteltes Dictionary hat uns sehr gut geholfen, den Überblick zu wahren.

### Negativ

+ Die schlechten Ergebnisse bei der Modellierung der neuronalen Netze und unsere Erkenntnis, dass dies wohl an der Datenmenge und den wenigen vergangenen Weihnachten liegt, hat eine fatalistische Grundstimmung befördert.
    + Dies könnte vor allem daran liegen, dass das erste Modell durch reinen Zufall ein exzellentes Ergebnis lieferte.
+ Zusätzlich verursachte der Fokus auf Data Preparation, dass wir sehr wenig Zeit hatten, um uns nochmals mit frischem Kopf auf die Modellierung über Regressoren zu stürzen.
+ Beide Umstände führten dazu, dass wir wenig besonnen manuelle Vergleiche von Ergebnissen mit verschiedenen Parametern anstellten, anstatt ein strukturiertes Vorgehen mit ```Pipelines``` und ```Grid-/Randomsearch``` zu wählen.

Um diese __Probleme in der Zukunft zu vermeiden__, schlagen wir folgendes Vorgehen vor:
+ Schnelle Exploration zu Beginn des Projekts
    + Die "tatsächliche" Größe des Datensatzes wäre uns so bewusst geworden und wir hätten die kommenden Probleme erwarten können.
    + iteratives Vorgehen statt alles brav nach Lehrbuch / CRISP-VGM
+ Dabei bereits automatisierte Evaluation
    + Die Nutzug von Pipelines und automatisierten Hyperparameter-Tuning-Verfahren hätte uns schnell die Wechselhaftigkeit der Ergebnisse gezeigt.
    + Dabei sollten neben den Ergebnissen auch Visualisierungen beispielsweise der ```learning_curve()``` genutzt werden.
+ Im Anschluss feste Meilensteine mit Deadlines für die CRISP-Phasen
    + Dies würde helfen, begeisterte Optimierungen in den ersten Phasen zu Lasten der späteren Phasen durchzuführen.

### Nächste Schritte

+ Dinge, die wir noch spannend gefunden hätten und gerne ermittelt hätten:
    + Ist eine "Deanonymisierung" der einzelnen Stores (beispielsweise über ihre Wetterdaten) im Bezug auf die ca 3000 Counties in den USA möglich? Ein erstes Herangehen wäre die "manuelle" Berechnung der euklidischen Distanz zwischen jedem Store und jedem County. Ein zweiter Schritt wäre die Nutzung von Clustering der Stores in durch die Counties vorgegebenen Cluster.
    + Vergleich über die Store-Department-Kombinationen hinweg un zu prüfen, ob es Kombinationen gibt, die signifikant bessere Ergebnisse liefern